In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import quantstream as qs
import os
import xarray as xr
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go

import sys

import dotenv


dotenv.load_dotenv("../.env")

sys.path.append("../")

FMP_API_KEY = os.getenv("FMP_API_KEY")

In [3]:
fmp = qs.FmpConnector(FMP_API_KEY)
data = fmp.get_intraday(
    symbol="NVDA",
    time_delta="5min",
    from_date="2023-02-01",
    to_date="2023-02-02",
)

In [4]:
data

<xarray.FinDataset> Size: 7kB
Dimensions:  (time: 156)
Coordinates:
  * time     (time) datetime64[ns] 1kB 2023-02-02T15:55:00 ... 2023-02-01T09:...
Data variables:
    open     (time) float64 1kB 21.66 21.51 21.5 21.51 ... 19.81 19.66 19.69
    low      (time) float64 1kB 21.62 21.5 21.47 21.45 ... 19.79 19.65 19.61
    high     (time) float64 1kB 21.75 21.67 21.53 21.56 ... 19.89 19.87 19.88
    close    (time) float64 1kB 21.72 21.66 21.5 21.5 ... 19.79 19.81 19.67
    volume   (time) int64 1kB 1499358 911354 424926 ... 1244523 1444246 1818253
Attributes:
    time_delta:  5min
    from_date:   2023-02-01
    to_date:     2023-02-02

In [ ]:
df = data.to_dataframe()
df

In [ ]:
data_intraday = fmp.get_intraday(
    symbol="AAPL",
    time_delta="5min",
    from_date="2023-02-01",
    to_date="2023-02-02",
)
data_intraday.to_dataframe()

In [ ]:
df["close"].plot()

In [ ]:
portfolio = qs.Portfolio()
portfolio.add_security("NVDA")
portfolio.add_security("AAPL")
portfolio.add_security("TSLA")
portfolio.add_security("AMZN")

In [ ]:
ds = portfolio.data["adjClose"]

In [ ]:
df = ds.to_dataframe()

In [ ]:
# put securities as columns
df = df.unstack(level=0)

In [ ]:
# drop the multiindex
df.columns = df.columns.droplevel()

In [ ]:
df.plot()

In [ ]:
df

In [ ]:
def generate_timeseries_json(df):
    json_data = {}
    for column in df.columns:
        json_data[column] = []
        for timestamp, value in df[column].iteritems():
            json_data[column].append(
                {"timestamp": timestamp.isoformat(), "value": value}
            )
    return json_data

 ### Changes Over Time

 - absolute changes
 - percentage change
 - log returns

In [ ]:
vals = ds.values

In [ ]:
df

In [ ]:
from quantstream import absolute_returns, percentage_returns, log_returns

### Log Returns, Absolute Returns, Percent Returns

In [ ]:
log_rets = log_returns(prices=df)
percentage_returns = percentage_returns(prices=df)
absolute_returns = absolute_returns(prices=df)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots with 3 rows and 1 column, not sharing the x-axis
fig = make_subplots(
    rows=3,
    cols=1,
    shared_xaxes=True,
    subplot_titles=("Log Returns", "Percentage Returns", "Absolute Returns"),
)

# Add log returns trace to the first row
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=log_rets["NVDA"],
        mode="lines",
        name="Log Returns",
        legendgroup="Log",
    ),
    row=1,
    col=1,
)

# Add percentage returns trace to the second row
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=percentage_returns["NVDA"],
        mode="lines",
        name="Percentage Returns",
        legendgroup="Percentage",
    ),
    row=2,
    col=1,
)

# Add absolute returns trace to the third row
fig.add_trace(
    go.Scatter(
        x=df.index,
        y=absolute_returns["NVDA"],
        mode="lines",
        name="Absolute Returns",
        legendgroup="Absolute",
    ),
    row=3,
    col=1,
)

# Separate legends for each subplot
fig.update_layout(
    showlegend=True,
    height=900,
    title_text="NVDA Returns: Log, Percentage, and Absolute",
)

# Position the legends outside the subplots and for each trace
fig.update_traces(showlegend=True)

# Update each trace to show individual legends
fig.update_layout(
    legend_tracegroupgap=50,
    height=900,
)

fig.show()